# Imports

In [ ]:
% pip install bs4 requests pandas

# Extract Links

In [16]:
from bs4 import BeautifulSoup
import requests
import pandas as pd



PAGENUMBER = 2
url = "https://www.namasha.com/channel7117104301"
all_links = []

for i in range(PAGENUMBER):
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')

    video_ids = [i.get("data-id") for i in soup.find_all("div", attrs={"class": "thumbnail"})]
    video_links = [i.get("href") for i in soup.find_all("a", attrs={"class": "thumbnail-title thumbnail-url flex-shrink-1 stretched-link"})]
    all_links.append(video_links)
    url = f"{url}?page={i}&qm=&lastId={video_ids[-1]}"


urls = []
for i in all_links:
    for j in i:
        urls.append(j)
print(len(urls))

120


# Get Information of links

In [ ]:
df = pd.DataFrame()
urls = urls[:65]

for url in urls:

    # url = "https://www.namasha.com/v/uRrRaQ7F"
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')

    title = soup.find("h1", attrs={"class": "video-title font-size-base font-size-lg-lg font-weight-bold mb-1"}).text.strip()
    link  = soup.find("span", attrs={"class": "video-like-count"}).text
    count_comment = soup.find("h4", attrs={"class": "font-size-base font-size-lg-sm mb-0"}).text.strip()
    time = soup.find("time" , attrs={"class", "text-dynamic-half-dark font-size-xs font-weight-light font-weight-lg-medium mt-2"}).text.strip()
    
    data = {
        'titles': [title],
        'likes': [link],
        "comments": [count_comment],
        "times":[time],
        "urls" :url}
    

    df2 = pd.DataFrame(data)
    df = pd.concat([df, df2], ignore_index=True)


# Store Dataset
FILENAME = f"namasha_channel{len(urls)}.csv"
df.to_csv(FILENAME)

# Cleaning dataset

In [42]:
# Cleaning
def clean_comments(x):
    rm_list = ["(" ,")", "نظرات"]

    for trash_words in rm_list:
        if trash_words in x:
            x = x.replace(trash_words , "")
    

    if x == "":
        return 0
    
    return int(x)
    

In [64]:
FILENAME = "namasha_channelas.csv"
df = pd.read_csv(FILENAME)
df["likes"] = df["likes"].apply(lambda x: int(x))
df["comments"] = df["comments"].apply(lambda x:clean_comments(x))

In [66]:
from IPython.display import display, HTML
# pd.set_option('display.max_columns', 90)

df = df.sort_values("likes", ascending=False)
display(HTML(df.to_html()))
df.to_csv("namasha.csv")

,Unnamed: 0,titles,likes,comments,times,urls
65,65,انیمه جاودانه شورشی XIAN NI (Renegade Immortal) قسمت 85,231,43,منتشر شده در تاریخ ۱۴۰۴/۰۱/۳۱,https://www.namasha.com/v/uYYn6cq9
190,190,انیمه شروع پس از پایان / آغاز پس از پایان The Beginning After the End قسمت 2,215,40,منتشر شده در تاریخ ۱۴۰۴/۰۱/۲۰,https://www.namasha.com/v/uYYn6cq9
166,166,انیمه نبرد از طریق اسمان Battle Through the Heavens فصل 5 قسمت 142,212,59,منتشر شده در تاریخ ۱۴۰۴/۰۱/۲۳,https://www.namasha.com/v/uYYn6cq9
109,109,انیمه شروع پس از پایان / آغاز پس از پایان The Beginning After the End قسمت 3,208,35,منتشر شده در تاریخ ۱۴۰۴/۰۱/۲۸,https://www.namasha.com/v/uYYn6cq9
35,35,انیمه شروع پس از پایان / آغاز پس از پایان The Beginning After the End قسمت 4,195,19,منتشر شده در تاریخ ۱۴۰۴/۰۲/۰۳,https://www.namasha.com/v/uYYn6cq9
208,208,انیمه جاودانه شورشی XIAN NI (Renegade Immortal) قسمت 83,195,49,منتشر شده در تاریخ ۱۴۰۴/۰۱/۱۷,https://www.namasha.com/v/uYYn6cq9
223,223,"انیمه از یه دهاتی پیر تا استاد شمشیرزنی Katainaka no Ossan, Kensei ni Naru قسمت 1",188,15,منتشر شده در تاریخ ۱۴۰۴/۰۱/۱۶,https://www.namasha.com/v/uYYn6cq9
83,83,انیمه نبرد از طریق اسمان Battle Through the Heavens فصل 5 قسمت 143,185,47,منتشر شده در تاریخ ۱۴۰۴/۰۱/۳۱,https://www.namasha.com/v/uYYn6cq9
191,191,انیمه قدیسهٔ زیادی بی‌نقص: نامزدم کنارم گذاشت و به یه فرمانروایی دیگه فروخته شدم قسمت 2,185,25,منتشر شده در تاریخ ۱۴۰۴/۰۱/۲۰,https://www.namasha.com/v/uYYn6cq9
222,222,انیمه نبرد از طریق اسمان Battle Through the Heavens فصل 5 قسمت 141,183,37,منتشر شده در تاریخ ۱۴۰۴/۰۱/۱۶,https://www.namasha.com/v/uYYn6cq9
